In [68]:
import sys
sys.path.append('code/cpet_articles/gathering/pdf_download_code')
from crossref_pdf_download import crossref_pdf_download
import pandas as pd
from tqdm import tqdm
import json
import requests

In [64]:
wiley_non_oa_status_codes = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_non_oa_status_codes.csv')
non200 = wiley_non_oa_status_codes[wiley_non_oa_status_codes['publisher_status_code'] != 200]
non200.shape

(1137, 52)

In [65]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = 'data/cpet_articles/pdfs/cdn_non_oa_pdfs'

with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/pdf_download_code/wiley_config.json') as config_file:
    wiley_token = json.load(config_file)['api_key']

In [71]:
test_doi = non200.loc[0,'doi']
test_doi

crossref_pdf_download(
    doi=test_doi,
    accept=accept,
    dest=dest_folder,
    user_agent=user_agent,
    TDM_token=crossref_token,
    TDM_header='Wiley-TDM-Client-Token',
    verify=True)

Status code 403 for DOI 10.1111/cpf.12525


{'doi': '10.1111/cpf.12525',
 'CR_status_code': 200,
 'publisher_status_code': 403}

In [72]:
doi = non200.loc[0,'doi_url']
doi_url = 'https://doi.org/' + str(doi)
out = {'doi': doi}
verify=True
TDM_header='Wiley-TDM-Client-Token'
TDM_token=crossref_token

crossref_headers = {'Accept': accept, 'User-Agent': user_agent}

In [73]:
crossref_response = requests.get(
    url = doi_url,
    headers=crossref_headers,
    allow_redirects=True,
    verify=verify)

out.update({'CR_status_code': crossref_response.status_code})
out

{'doi': 'https://doi.org/10.1111/cpf.12525', 'CR_status_code': 200}

In [74]:
crossref_response.json()['link'][0]

{'URL': 'https://api.wiley.com/onlinelibrary/tdm/v1/articles/10.1111%2Fcpf.12525',
 'content-type': 'application/pdf',
 'content-version': 'vor',
 'intended-application': 'text-mining'}

In [42]:
full_text_link = crossref_response.json()['link'][0]['URL'] # change to full-text xml link later

In [41]:
if 'application/xml' in crossref_response.json()['link'][0]['content-type']:
    print('xml is in there')

xml is in there


In [58]:
publisher_headers = {'User-Agent': user_agent, 'Content-Type': 'application/xml'}
if (TDM_header != None) & (TDM_token != None):
    publisher_headers.update({TDM_header: TDM_token})
publisher_headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0',
 'Content-Type': 'application/xml',
 'CR-TDM-Client-Token': 'f95f9ecf-68f16d7d-ac324cc4-8e1da41d'}

In [59]:
publisher_response = requests.get(full_text_link, headers = publisher_headers, allow_redirects=True)
publisher_response.status_code

403

In [ ]:
publisher_headers = {'User-Agent': user_agent}
if (TDM_header != None) & (TDM_token != None):
    publisher_headers.update({TDM_header: TDM_token})

publisher_response = requests.get(pdf_url, headers = publisher_headers, allow_redirects=True, verify=verify)
out.update({'publisher_status_code': publisher_response.status_code})

if publisher_response.status_code != 200:
    print(f'Status code {publisher_response.status_code} for DOI {doi}')
    return out

doi_suffix = str(doi.split('/')[1:]).strip("[']")
filename = f'{dest}/{doi_suffix}.pdf'

with open(filename, mode='wb') as f:
    f.write(publisher_response.content)

except Exception as e:
print(f'Exception at DOI {doi}')
print(e)
out.update({'error': e})

return out

In [ ]:
res = []
for i, row in tqdm(cdn.iterrows(), total=cdn.shape[0]):
    temp = crossref_pdf_download(
        doi=row['doi'],
        accept=accept,
        dest=dest_folder,
        user_agent=user_agent,
        verify=True)
    res.append(temp)